In [1]:
import pandas as pd
import numpy as np
import os
import re

In [6]:
date = "20210709_204655"
directory = f'_output/hedges/pnl/{date}'

In [7]:
PNL = "PNL__KDE__DeltaGammaHedge__BULLISH__3000__30.csv"

In [8]:
prices = pd.read_csv("_output/hedges/pnl/20210709_204655/__prices.csv")
prices = dict(prices[["label", "price"]].to_numpy())

In [47]:
df[(df['paths'] == 'SVCJ') & (df['period'] == 'CALM')].shape

(92, 12)

In [58]:
len([1 for r in rows if r[0] == 'SVCJ' and r[3] == 'CALM'])

138

In [62]:
df[:20]

,paths,model,hedge,period,strike,maturity,mean,std,skew,kurt,1%,99%
0,KDE,SVCJ,DeltaHedge,COVID,9000,30,0.105,0.180,-0.000,0.001,-0.372,0.527
1,KDE,MERTON,DeltaVegaHedge,BULLISH,3000,90,0.054,0.372,0.001,0.003,-0.842,1.208
2,SVCJ,BLACK_SCHOLES,DeltaGammaHedge,BULLISH,3000,90,0.013,0.113,-0.000,0.069,-0.310,0.293
3,KDE,SVCJ,DeltaVegaHedge,CALM,7000,90,0.038,0.076,0.000,0.001,-0.148,0.237
4,SVCJ,SVJ,DeltaHedge,CALM,7000,30,-0.004,0.377,-0.001,0.004,-1.408,0.686
11,KDE,HESTON,DeltaVegaHedge,COVID,9000,30,-0.016,0.176,-0.000,0.001,-0.490,0.401
12,SVCJ,MERTON,MinimumVarianceHedge,BULLISH,3000,30,0.002,0.101,0.002,0.048,-0.261,0.225
14,KDE,CGMY,DeltaHedge,CALM,7000,30,0.157,0.151,0.000,0.000,-0.191,0.538
15,KDE,SVJ,MinimumVarianceHedge,CALM,9000,30,0.577,0.327,0.001,0.007,-0.352,1.610
18,SVCJ,CGMY,DeltaHedge,CALM,7000,90,-0.016,0.365,-0.001,0.008,-1.159,0.741


In [68]:
rows = []

# stats = ['mean', 'std', 'skew', 'kurt', '1%', '25%', '50%', '75%', '99%']
stats = ['mean', 'std', 'skew', 'kurt', '1%', '99%']

for fname in [fname for fname in os.listdir(directory) if fname.endswith('.csv') and fname.startswith('PNL')]:
    paths, model, hedge, period, strike, maturity = re.match("PNL__(\w+)__(\w+)__(\w+)__(\w+)__(\w+)__(\w+)__100000.csv", fname).groups()
    df = pd.read_csv(f"{directory}/{fname}", header=None)
    label = re.match(".*_(\w+__\d+__\d+)__\d+.csv$", fname).groups()[0]
    price = prices[label]
    quantile_low = df.quantile(0.01)
    quantile = df.quantile(.99)
    skew = df.skew()
    kurt = df.kurtosis()
    df = df.describe()
    df = df.T
    df['skew'] = skew
    df['kurt'] = kurt
    df['1%'] = quantile_low
    df['99%'] = quantile
    df = np.round(df/price, 3)
    del df['count']
    del df['min']
    del df['max']
    df = df[stats]
    rows += [[paths, model, hedge, period, strike, maturity, *df.to_numpy()[0]]]
    
columns = ['paths', 'model', 'hedge', 'period', 'strike', 'maturity', *stats]
df = pd.DataFrame(rows, columns=columns)

# df = df[df['maturity'] == '90']
df = df[df['strike'].isin(['8367', '9804', '4088'])]
df

,paths,model,hedge,period,strike,maturity,mean,std,skew,kurt,1%,99%
5,SVCJ,HESTON,DeltaGammaHedge,COVID,9804,30,-0.118,0.587,-0.006,0.104,-2.048,1.242
6,KDE,SVJ,DeltaHedge,BULLISH,4088,30,-0.301,0.632,0.001,0.009,-1.960,1.476
7,SVCJ,MERTON,MinimumVarianceHedge,COVID,9804,30,-0.075,1.329,-0.004,0.010,-5.127,1.733
8,KDE,BLACK_SCHOLES,DeltaHedge,CALM,8367,90,0.466,0.136,0.000,0.000,0.131,0.796
10,SVCJ,SVJ,DeltaGammaHedge,COVID,9804,30,-0.135,0.605,-0.004,0.079,-2.161,1.282
...,...,...,...,...,...,...,...,...,...,...,...,...
809,SVCJ,SVCJ,DeltaVegaHedge,COVID,9804,30,-0.127,0.639,-0.002,0.085,-2.370,1.280
810,KDE,VARIANCE_GAMMA,DeltaHedge,BULLISH,4088,30,-0.302,0.634,0.001,0.009,-1.973,1.484
814,SVCJ,BLACK_SCHOLES,DeltaGammaHedge,CALM,8367,30,0.014,0.438,0.003,0.043,-1.216,1.250
820,KDE,MERTON,DeltaHedge,CALM,8367,30,0.432,0.260,-0.000,0.003,-0.296,1.120


In [70]:
hs = {
    'DeltaHedge': 0,
    'DeltaGammaHedge': 1,
    'DeltaVegaHedge': 2,
    'MinimumVarianceHedge': 3,
}

ms = {
    'BLACK_SCHOLES': 0,
    'HESTON': 1,
    'VARIANCE_GAMMA': 2,
    'SVJ': 3,
    'SVCJ': 4,
    'CGMY': 5,
    'MERTON': 6,
}

hn = {
    'DeltaHedge': "\Delta ",
    'DeltaGammaHedge': "\Delta-\Gamma ",
    'DeltaVegaHedge': "\Delta-\mathcal V ",
    'MinimumVarianceHedge': "\textbf{MV} ",
}

mn = {
    'BLACK_SCHOLES': "BS",
    'HESTON': "SV",
    'VARIANCE_GAMMA': "VG",
    'SVJ': 'SVJ',
    'SVCJ': 'SVCJ',
    'CGMY': 'CGMY',
    'MERTON': 'Merton',
}

for (paths, period), odf in df.groupby(['paths', 'period']):
    print("# ", paths, period, "\n\n")
    odf1 = odf[odf['maturity'] == '30']
    odf2 = odf[odf['maturity'] == '90']
    odf = pd.merge(odf1, odf2, on=['paths', 'model', 'hedge', 'period', 'strike'], suffixes=('_30', '_90'))
    odf['hs'] = odf.apply(lambda x: hs[x['hedge']], axis=1)
    odf['ms'] = odf.apply(lambda x: ms[x['model']], axis=1)
    odf['name'] = odf.apply(lambda x: f"${hn[x['hedge']]}_{{{mn[x['model']]}}}$", axis=1)
    odf = np.round(odf, 3)
    odf = odf.sort_values(['hs', 'ms'])[['name', *[f'{x}_30' for x in stats], *[f'{x}_90' for x in stats]]]
#     odf.columns = ['name', 'mean', 'std', 'skew', 'kurt', '1\%', '25\%', '50\%', '75\%', '99\%']
    caption = f"Summary statistics of {period} {paths} hedges"
    label = f"tab:{paths}_{period}"
    print(f"\\begin{{table}}\n\centering\n{odf.to_latex(index=False, escape=False)}\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\end{{table}}")
    print('\n==================================================================\n\n')

#  KDE BULLISH 


\begin{table}
\centering
\begin{tabular}{lrrrrrrrrrrrr}
\toprule
                          name &  mean_30 &  std_30 &  skew_30 &  kurt_30 &  1%_30 &  99%_30 &  mean_90 &  std_90 &  skew_90 &  kurt_90 &  1%_90 &  99%_90 \\
\midrule
                $\Delta _{BS}$ &   -0.304 &   0.631 &    0.000 &    0.009 & -1.972 &   1.465 &   -0.170 &   1.019 &    0.001 &    0.010 & -2.738 &   2.960 \\
                $\Delta _{SV}$ &   -0.300 &   0.632 &    0.001 &    0.009 & -1.959 &   1.476 &   -0.177 &   1.020 &    0.001 &    0.010 & -2.754 &   2.953 \\
                $\Delta _{VG}$ &   -0.302 &   0.634 &    0.001 &    0.009 & -1.973 &   1.484 &   -0.170 &   1.019 &    0.001 &    0.010 & -2.735 &   2.963 \\
               $\Delta _{SVJ}$ &   -0.301 &   0.632 &    0.001 &    0.009 & -1.960 &   1.476 &   -0.178 &   1.020 &    0.001 &    0.010 & -2.754 &   2.950 \\
              $\Delta _{SVCJ}$ &   -0.299 &   0.632 &    0.001 &    0.009 & -1.955 &   1.479 &   -0.173 &   1.020 &   

In [71]:
for fname in [fname for fname in os.listdir(".") if fname.endswith('4000__30__100000.csv') and fname.startswith('PNL__SVCJ__BLACK_SCHOLES')]:
    df = pd.read_csv(f"{fname}", header=None)
    label = re.match(".*_(\w+__\d+__\d+)__\d+.csv$", fname).groups()[0]
    price = prices[label]
    print(price)
    quantile_low = df.quantile(0.01)
    quantile = df.quantile(.99)
    skew = df.skew()
    kurt = df.kurtosis()
    df = df.describe()
    df = df.T
    df['skew'] = skew
    df['kurt'] = kurt
    df['1 %'] = quantile_low
    df['99 %'] = quantile
    df = np.round(df/price, 4)
    del df['count']
    del df['min']
    del df['max']
    df = df[['mean', 'std', 'skew', 'kurt','1 %','25%','50%','75%','99 %']]
    print(fname)
    display(df)
    print("\n\n")
    print(df.to_latex())
    print("\n\n")

In [20]:
os.getcwd()

'/Users/lilimatic/chris/chris/_output/hedges/pnl/20210228_021905'

In [21]:
df

,mean,std,skew,kurt,1 %,25%,50%,75%,99 %
0,0.0279,0.5143,0.0189,0.6854,-1.4177,-0.066,0.0808,0.2167,1.0001
